# Métodos iterativos creados a mano

In [1]:
using LinearAlgebra, SparseArrays, BenchmarkTools
#using CUDA, CUDA.CUSPARSE

## Jacobi secuencial

### Matricial

In [26]:
function vectorizedjacobi(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= D⁻¹(L+U)x⁽ᵏ⁻¹⁾+D⁻¹b
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    # Definimos a N como la suma de estas dos últimas matrices.
    D = Diagonal(A)
    N = - triu(A,1) - tril(A,-1)
    x = rand(length(b))
    while norm(A*x - b) > ϵ
        x = D\(N*x + b)
    end
    return Vector(x)
end

vectorizedjacobi (generic function with 4 methods)

### Paralelo

In [3]:
function paralleljacobi(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    D = diag(A);
    D = CuArray{Float64}(D)
    N = - triu(A,1) - tril(A,-1)
    N = CuSparseMatrixCSR{Float64}(N)
    
    A = CuSparseMatrixCSR{Float64}(A)
    b = CuArray{Float64}(b)
    x = CUDA.rand{Float64}(length(b))
    while norm(A*x - b) > ϵ
        x = D\(N*x + b)
    end
    return x
end

paralleljacobi (generic function with 2 methods)

---

## Gauss-Seidel

### Matricial

In [4]:
function vectorizedgaussseidel(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-L)⁻¹(Ux⁽ᵏ⁻¹⁾+b)
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    U = - triu(A,1) 
    L₀ = tril(A,0)
    x = rand(length(b))
    while norm(A*x - b) > ϵ
        x = L₀\(U*x + b)
    end
    return x
end

vectorizedgaussseidel (generic function with 2 methods)

### Paralelo

In [5]:
function parallelgaussseidel(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-L)⁻¹(Ux⁽ᵏ⁻¹⁾+b)
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    U = - triu(A,1) 
    L₀ = tril(A,0)
    U = CuSparseMatrixCSR{Float64}(U)
    L₀ = CuSparseMatrixCSR{Float64}(L₀)

    A = CuSparseMatrixCSR{Float64}(A)
    b = CuArray{Float64}(b)
    x = CUDA.rand{Float64}(length(b))
    while norm(A*x - b) > ϵ
        x = L₀\(U*x + b)
    end
    return x
end

parallelgaussseidel (generic function with 2 methods)

---

## SOR (Successive Overrelaxation)

### Matricial

In [6]:
function vectorizedsor(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ω = 1.8::Float64, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-ωL)⁻¹(ωU+(1-ω)D)x⁽ᵏ⁻¹⁾+ω(D-ωL)⁻¹b
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    # ω es justo un argumento de sobrerelajación, que sirve para hacer más rápido al método. Hay que escogerlo 
    # ente (0,1)
    D = Diagonal(A)
    U = - triu(A,1) 
    L = - tril(A,-1)
    x = rand(length(b))
    
    L′ = D-ω*L
    U′ = ω*U+(1.0-ω)*D
    b′ = ω*b
    while norm(A*x - b) > ϵ
        x = L′ \ (U′*x + b′)
    end
    return x
end

vectorizedsor (generic function with 3 methods)

### Paralelo

In [7]:
function parallelsor(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ω = 1.8::Float64, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-L)⁻¹(Ux⁽ᵏ⁻¹⁾+b)
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    D = Diagonal(A)
    D = CuArray{Float64}(D)
    U = - triu(A,1) 
    L = - tril(A,-1)
    U = CuSparseMatrixCSR{Float64}(U)
    L = CuSparseMatrixCSR{Float64}(L)

    A = CuSparseMatrixCSR{Float64}(A)
    b = CuArray{Float64}(b)
    x = CUDA.rand{Float64}(length(b))
    
    L′ = D-ω*L
    U′ = ω*U+(1.0-ω)*D
    b′ = ω*b
    while norm(A*x - b) > ϵ
        x = L′ \ (U′*x + b′)
    end
    return x
end

parallelsor (generic function with 3 methods)

---

## GMRES

### Normalito

In [8]:
function leastsquares(H::Matrix{Float64}, r)
    r′ = zeros(size(H)[1])
    r′[1] = norm(r)
    x = H \ r′
end

leastsquares (generic function with 1 method)

In [9]:
# Algoritmo sacado directo del pseudocódigo
function gmres(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
        end
        k += 1
    end
    return x
end

gmres (generic function with 2 methods)

### Normalito GPU

In [28]:
# Algoritmo sacado directo del pseudocódigo
function gmres(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
        end
        k += 1
    end
    return x
end

gmres (generic function with 2 methods)

### Reiniciado

In [10]:
function reiniciarvariables(x::Vector, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64})
    k = 1
    x₀ = x
    r = b - A*x₀
    H = zeros(2,1)
    q = [r / norm(r)]
    return x₀, r, q, k, H
end
    
function gmresreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        #if k % 25 == 0 # cuando tenemos 30³ nuestra matriz ( Q = 5,400,000)
        if k % 25 == 0 # cuando tenemos 40³ nuestra matriz ( Q = 10,240,000)
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
            x₀, residual₀, q, k, H = reiniciarvariables(x, A, b)
        end
    end
    return x
end

gmresreiniciado (generic function with 2 methods)

### Reiniciado GPU

In [10]:
function reiniciarvariables(x, A, b)
    k = 1
    x₀ = x
    r = b - A*x₀
    H = zeros(2,1)
    q = [r / norm(r)]
    return x₀, r, q, k, H
end
    
function gmresreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        #if k % 25 == 0 # cuando tenemos 30³ nuestra matriz ( Q = 5,400,000)
        if k % 25 == 0 # cuando tenemos 40³ nuestra matriz ( Q = 10,240,000)
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
            x₀, residual₀, q, k, H = reiniciarvariables(x, A, b)
        end
    end
    return x
end

gmresreiniciado (generic function with 2 methods)

### Precondicionado

In [11]:
function gmresprecondicionado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = (b - A*x₀)
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
        end
        k += 1
    end
    return x
end

gmresprecondicionado (generic function with 2 methods)

### Precondicionado GPU

In [11]:
function gmresprecondicionado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = (b - A*x₀)
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
        end
        k += 1
    end
    return x
end

gmresprecondicionado (generic function with 2 methods)

### Precondicionado y reiniciado

In [12]:
function reiniciarvariablesprecondicionado(x::Vector, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        Ms::Vector)
    k = 1
    x₀ = x
    H = zeros(2,1) 
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    return x₀, residual₀, q, k, H
end

# Algoritmo sacado directo del pseudocódigo
function gmresprecondicionadoreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, 
        ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        # if k % 20 == 0 # Para 40³ sirvió chido el módulo 20 para el de Jacobi
        # if k % 25 == 0 # Para 40³ sirvió chido el módulo 25 para el de Gauss-Seidel
        if k % 30 == 0 # Para 40³ sirvió chido el módulo 30 para el de SOR
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
            x₀, residual₀, q, k, H = reiniciarvariablesprecondicionado(x, A, b, Ms)
        end
    end
    return x
end

gmresprecondicionadoreiniciado (generic function with 2 methods)

### Precondicionado y reiniciado GPU

In [12]:
function reiniciarvariablesprecondicionado(x, A, b, Ms::Vector)
    k = 1
    x₀ = x
    H = zeros(2,1)
    
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    return x₀, residual₀, q, k, H
end

# Algoritmo sacado directo del pseudocódigo
function gmresprecondicionadoreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, 
        ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        # if k % 20 == 0 # Para 40³ sirvió chido el módulo 20 para el de Jacobi
        # if k % 25 == 0 # Para 40³ sirvió chido el módulo 25 para el de Gauss-Seidel
        if k % 30 == 0 # Para 40³ sirvió chido el módulo 30 para el de SOR
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
            x₀, residual₀, q, k, H = reiniciarvariablesprecondicionado(x, A, b, Ms)
        end
    end
    return x
end

gmresprecondicionadoreiniciado (generic function with 2 methods)

---

In [13]:
include("../notebooks/FVM.jl")
include("Solvers.jl")

function Γ_constant(x::Array, y::Array, z::Array)
    Γ = 1000
    tensor_Γ = ones(length(x), length(y), length(z))
    tensor_Γ = Γ .* tensor_Γ
end

function getsystem(vol)
    # Construcción de la malla.
    volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces = FVM.uniform_grid(vol, vol/10)
    tags = FVM.init_tags(3, volumes, centers_and_boundaries)
    tags_b = FVM.init_tags_boundaries(3, centers_and_boundaries)
    FVM.tag_wall(tags, tags_b, [:W, :E, :T, :N, :B], 0, :D)
    FVM.tag_wall(tags, tags_b, :S, 100, :D)
    mesh = FVM.Mesh(volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces, tags, tags_b)
    # Generando los coeficientes
    coeff = FVM.init_coefficients(mesh)
    FVM.set_diffusion(coeff, Γ_constant);
    # Construyendo el sistema de ecuaciones
    equation_system = FVM.init_eq_system(coeff)
    A = equation_system.A
    b = equation_system.b
    return A, b
end

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/oscar/.julia/packages/Plots/9C6z9/src/backends.jl:432


getsystem (generic function with 1 method)

## Pruebas

In [14]:
A, b = getsystem(40);

In [27]:
@btime x = vectorizedjacobi(A, b)

  14.309 s (238416 allocations: 38.42 GiB)


64000-element Vector{Float64}:
  0.36769248623120987
  0.06957996274120772
  0.018272872168189357
  0.006866879523603204
  0.0038847756193864306
  0.0030442924986847208
  0.0028670225918238203
  0.0029302434558816557
  0.0030879062809870796
  0.0032808614415265167
  0.0034819112748014015
  0.003677177586188068
  0.0038589193855359436
  ⋮
 41.70576307696519
 41.70023977758334
 41.69277691167693
 41.682473336442044
 41.667822430784064
 41.646152396990196
 41.61234019351861
 41.55542959128582
 41.44796771686943
 41.20428400202758
 40.479401828249316
 32.238737880318325

In [16]:
@btime x = vectorizedgaussseidel(A, b)

  6.881 s (32344 allocations: 7.72 GiB)


64000-element Vector{Float64}:
  0.3676924862307518
  0.06957996273990295
  0.018272872166154266
  0.00686687952088029
  0.003884775616022536
  0.0030442924947276014
  0.002867022587326135
  0.002930243450895332
  0.003087906275568915
  0.0032808614357308333
  0.0034819112686872164
  0.0036771775798102534
  0.0038589193789538025
  ⋮
 41.70576307697095
 41.700239777588834
 41.69277691168212
 41.682473336446876
 41.6678224307885
 41.646152396994175
 41.61234019352211
 41.555429591288785
 41.44796771687181
 41.204284002029354
 40.47940182825045
 32.23873788031872

In [17]:
α = 1.79
@btime x = vectorizedsor(A, b, α)

  793.392 ms (3604 allocations: 891.01 MiB)


64000-element Vector{Float64}:
  0.3676924862210351
  0.06957996272060243
  0.01827287213913775
  0.0068668794842219275
  0.003884775572470832
  0.003044292441846941
  0.0028670225361121927
  0.0029302433816329756
  0.0030879062223006597
  0.003280861355604598
  0.0034819112111689754
  0.003677177500762443
  0.0038589193109474307
  ⋮
 41.70576307699985
 41.70023977761602
 41.69277691170741
 41.68247333647011
 41.66782243080949
 41.646152397012784
 41.61234019353821
 41.55542959130226
 41.447967716882566
 41.204284002037355
 40.47940182825545
 32.23873788032091

In [18]:
@btime x = gmres(A, b)

  5.625 s (116190 allocations: 24.73 GiB)


64000-element Vector{Float64}:
  0.367692486247971
  0.06957996275296596
  0.018272872186998412
  0.006866879537238857
  0.0038847756341026107
  0.0030442925186684733
  0.002867022618155801
  0.002930243482789474
  0.0030879063215644154
  0.0032808614656822666
  0.0034819113075943153
  0.003677177636582929
  0.003858919420762663
  ⋮
 41.70576307701417
 41.70023977762857
 41.69277691171398
 41.682473336482076
 41.667822430811434
 41.64615239701158
 41.612340193530486
 41.55542959130928
 41.44796771688486
 41.20428400204046
 40.47940182824584
 32.23873788031762

In [19]:
@btime x = gmresreiniciado(A, b)

  1.796 s (52372 allocations: 6.49 GiB)


64000-element Vector{Float64}:
  0.3676924862969842
  0.06957996271109973
  0.01827287216841044
  0.006866879547972044
  0.0038847755970128697
  0.0030442925125057797
  0.0028670226216769355
  0.00293024342738091
  0.003087906298840966
  0.003280861482179636
  0.0034819112568251335
  0.0036771775925675478
  0.003858919423051637
  ⋮
 41.70576307697801
 41.700239777633406
 41.69277691173014
 41.68247333647123
 41.667822430820486
 41.64615239701454
 41.612340193529896
 41.5554295913319
 41.44796771690404
 41.204284001996015
 40.479401828244036
 32.23873788031301

In [20]:
M_jacobi = Diagonal(A)
@btime x = gmresprecondicionado(A, b, [M_jacobi])

  5.214 s (184432 allocations: 34.15 GiB)


64000-element Vector{Float64}:
  0.3676924870178751
  0.06957996446599679
  0.018272872404660023
  0.00686687823437615
  0.003884777520572029
  0.003044296552566106
  0.002867027182916815
  0.002930245888317473
  0.003087905819055381
  0.0032808624244288487
  0.0034819118032168306
  0.003677178341768686
  0.0038589205737453725
  ⋮
 41.705763078705004
 41.70023977938506
 41.692776912038646
 41.682473334394466
 41.66782242455828
 41.646152392252795
 41.61234018991697
 41.555429589562166
 41.44796771595145
 41.204284000669716
 40.479401828286
 32.238737880583116

In [21]:
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_gauss_seidel_1 = I+(L*inv(D))
M_gauss_seidel_2 = D+U
@btime x = gmresprecondicionado(A, b, [M_gauss_seidel_1, M_gauss_seidel_2])

  534.084 ms (15865 allocations: 1.99 GiB)


64000-element Vector{Float64}:
  0.36769249008085303
  0.06957996490213036
  0.018272873009834767
  0.006866878732332351
  0.003884776252704447
  0.0030442941320021344
  0.0028670240605898067
  0.0029302450339094843
  0.003087906733675816
  0.0032808638696719683
  0.003481912894808148
  0.003677177802776155
  0.0038589203676602235
  ⋮
 41.70576307731965
 41.70023977793387
 41.692776912412505
 41.68247333722133
 41.66782243179651
 41.646152398675376
 41.61234019484105
 41.55542959255295
 41.44796771731419
 41.204284006329935
 40.4794018393343
 32.23873787713783

In [22]:
α = 1.8
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_sor_1 = I+(α*L*inv(D))
M_sor_2 = D+α*U
@btime x = gmresprecondicionado(A, b, [M_sor_1, M_sor_2])

  186.342 ms (4503 allocations: 555.78 MiB)


64000-element Vector{Float64}:
  0.36769248499257146
  0.06957996226615798
  0.018272872679857777
  0.006866880065343736
  0.0038847759308600627
  0.003044292429662643
  0.002867022358723492
  0.0029302433533027195
  0.003087906264074493
  0.0032808613244246776
  0.003481910914399311
  0.0036771769077597882
  0.0038589185284556016
  ⋮
 41.70576307839104
 41.70023977880769
 41.69277691294756
 41.68247333789433
 41.66782243261602
 41.64615239963187
 41.61234019727254
 41.55542959581135
 41.44796771908348
 41.204283995321184
 40.47940180062696
 32.23873791738716

In [23]:
M_jacobi = Diagonal(A)
@btime x = gmresprecondicionadoreiniciado(A, b, [M_jacobi])

  2.096 s (69896 allocations: 12.41 GiB)


64000-element Vector{Float64}:
  0.3676924945785465
  0.06957996323921063
  0.018272867306111926
  0.006866864486577004
  0.0038847691202381087
  0.0030442959290037787
  0.002867013281052611
  0.0029302246205793006
  0.00308789764604213
  0.0032808632534090014
  0.0034819017617338825
  0.003677147778125971
  0.00385890350254804
  ⋮
 41.70576305436971
 41.70023975486283
 41.69277688861182
 41.68247331318429
 41.667822413846636
 41.64615238439366
 41.61234017792923
 41.5554295753687
 41.44796770529264
 41.20428399153333
 40.47940182048338
 32.23873787815478

In [24]:
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_gauss_seidel_1 = I+(L*inv(D))
M_gauss_seidel_2 = D+U
@btime x = gmresprecondicionadoreiniciado(A, b, [M_gauss_seidel_1, M_gauss_seidel_2])

  511.447 ms (11926 allocations: 1.47 GiB)


64000-element Vector{Float64}:
  0.3676924862317251
  0.06957996274473019
  0.01827287217815651
  0.006866879493802307
  0.0038847756112387073
  0.0030442924833424564
  0.0028670226042484728
  0.0029302434612393704
  0.003087906265369332
  0.003280861415444362
  0.003481911289025844
  0.0036771776179324277
  0.0038589194220483088
  ⋮
 41.705763076997215
 41.70023977761151
 41.69277691171003
 41.682473336460994
 41.6678224308075
 41.64615239701798
 41.61234019351828
 41.555429591299905
 41.447967716998726
 41.20428400187987
 40.479401828207216
 32.23873788035492

In [25]:
α = 1.8
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_sor_1 = I+(α*L*inv(D))
M_sor_2 = D+α*U
@btime x = gmresprecondicionadoreiniciado(A, b, [M_sor_1, M_sor_2])

  171.358 ms (3992 allocations: 504.79 MiB)


64000-element Vector{Float64}:
  0.3676924833863077
  0.06957996352325113
  0.018272874388519423
  0.006866880053989055
  0.003884774757820897
  0.0030442914993165093
  0.002867021604690434
  0.0029302425546432476
  0.003087905171865267
  0.0032808598857609894
  0.0034819091713231554
  0.0036771756029913005
  0.003858917876842227
  ⋮
 41.70576307294571
 41.70023977345858
 41.69277690724699
 41.6824733317479
 41.66782242603446
 41.646152393342334
 41.612340192959664
 41.55542959664685
 41.44796772555993
 41.20428399454414
 40.47940175598575
 32.238737909549954

---